# PostgreSQL

In [1]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine, text
from tabulate import tabulate
import os
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

# PostgreSQL Database Connection Details
POSTGRES_HOST = os.getenv("POSTGRES_LOCALHOST")
POSTGRES_PORT = os.getenv("POSTGRES_PORT", "5432")  # default Postgres port
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")

# Construct the database URI (using psycopg2 as driver)
DATABASE_URI = f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

# Create the SQLDatabase object
db = SQLDatabase.from_uri(DATABASE_URI)

In [13]:
# List all tables
tables = db.get_usable_table_names()
print(tables)

['bookings', 'guests', 'room_types', 'rooms']


In [14]:
engine = create_engine(DATABASE_URI)

# 3. Connect to the database and execute the query.
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM guests;"))
        rows = result.fetchall()
        headers = result.keys()

        # 4. Print the formatted result.
        print("Counting all guests in the database:")
        print(tabulate(rows, headers=headers, tablefmt="psql"))

except Exception as e:
    print(f"An error occurred: {e}")

Counting all guests in the database:
+---------+
|   count |
|---------|
|       2 |
+---------+


# BigQuery

In [2]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

In [ ]:
def list_bq_tables(project_id, dataset_id):
    """
    Lists all tables in a specified BigQuery dataset.

    Args:
        project_id (str): Your Google Cloud project ID.
        dataset_id (str): Your BigQuery dataset ID.
    """
    try:
        # 1. Construct a BigQuery client object.
        #    The client library will automatically use your authenticated
        #    credentials (from the gcloud CLI).
        client = bigquery.Client(project=project_id)

        # 2. Get a reference to the dataset.
        dataset_ref = client.dataset(dataset_id)

        print(f"Fetching tables for dataset '{dataset_id}' in project '{project_id}'...")

        # 3. Use the client to list the tables in the dataset.
        #    This returns an iterator, which we will loop through.
        tables = list(client.list_tables(dataset_ref))

        # 4. Check if any tables were found and print their names.
        if tables:
            print("Tables found:")
            for table in tables:
                # The .table_id attribute gives you the name of the table.
                print(f"- {table.table_id}")
        else:
            print(f"No tables found in dataset '{dataset_id}'.")

    except NotFound:
        print(f"Error: Dataset '{dataset_id}' not found.")
        print("Please check your project and dataset IDs.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        print("Please ensure you are authenticated correctly (see instructions).")


In [5]:
GCP_PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")
BIGQUERY_DATASET_ID = "hotel_analytics_dataset"
# -------------------

list_bq_tables(GCP_PROJECT_ID, BIGQUERY_DATASET_ID)

Fetching tables for dataset 'hotel_analytics_dataset' in project 'eikon-dev-ai-team'...
Tables found:
- booking_analytics
- hotels


In [5]:
import datetime

def get_current_date_wib() -> str:
    """Mendapatkan tanggal saat ini dalam Waktu Indonesia Barat (WIB) dan mengembalikannya sebagai string yang diformat."""
    # Tentukan zona waktu WIB (UTC+7)
    wib_timezone = datetime.timezone(datetime.timedelta(hours=7))
    
    # Dapatkan waktu saat ini di WIB
    now = datetime.datetime.now(wib_timezone)
    
    # Format tanggal menjadi string (contoh: "Thursday, September 18, 2025")
    return now.strftime('%Y-%m-%d')

In [4]:
get_current_date_wib()

'2025-09-19'